In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/License-super-resolution-master2/

/content/drive/MyDrive/License-super-resolution-master2


In [ ]:
from Utilities.io import DataLoader
from Utilities.lossMetric import *
from Utilities.trainVal import MinMaxGame
from Models.RRDBNet import RRDBNet
from Models.GAN import Discriminator

In [ ]:
import numpy as np
import glob
PATHTRAIN = '/content/drive/MyDrive/DatasetFinalPb/train/' # only use images with shape 192 by 96 for training
PATHTEST = '/content/drive/MyDrive/DatasetFinalPb/test/'
filestrain = glob.glob(PATHTRAIN + '/*.png') 
filestest = glob.glob(PATHTEST + '/*.png')
#print(list(files))
np.random.shuffle(filestrain)
np.random.shuffle(filestest)
train = filestrain
val = filestest
loader = DataLoader()
trainData = DataLoader().load(train, batchSize=40)
valData = DataLoader().load(val, batchSize=5)

### Training

In [ ]:
discriminator = Discriminator()
extractor = buildExtractor()
generator = RRDBNet(blockNum=10)

80142336/80134624 [==============================] - 1s 0us/step


*  It's a good idea to pretrain the generator model before the min-max game - Reference: https://arxiv.org/abs/1701.00160

In [ ]:
# a simple custom loss function that combines MAE loss with VGG loss, as defined in the SRGAN paper
def contentLoss(y_true, y_pred):
    featurePred = extractor(y_pred)
    feature = extractor(y_true)
    mae = tf.reduce_mean(tfk.losses.mae(y_true, y_pred))
    return 0.1*tf.reduce_mean(tfk.losses.mse(featurePred, feature)) + mae

optimizer = tfk.optimizers.Adam(learning_rate=1e-3)
generator.compile(loss=contentLoss, optimizer=optimizer, metrics=[psnr, ssim])
# epoch is set to 1 for demonstration purpose. In practice I found 20 is a good number
# When the model reaches PSNR=20/ssim=0.65, we can start the min-max game
history = generator.fit(x=trainData, validation_data=valData, epochs=1, steps_per_epoch=100, validation_steps=100)

InvalidArgumentError: ignored

### Generative adverserial network training

In [ ]:
# training parameter. epoch is set to 1 for demonstration
# please train the network utill it reaches snRatio ~= 22 
PARAMS = dict(lrGenerator = 1e-4, 
              lrDiscriminator = 1e-4,
              epochs = 1, 
              stepsPerEpoch = 100, 
              valSteps = 100)
game = MinMaxGame(generator, discriminator, extractor)
log, valLog = game.train(trainData, valData, PARAMS)
game.train(trainData, valData, PARAMS)
# ideally peak signal noise ratio(snRation or psnr) should reach ~22

ResourceExhaustedError: ignored

Saving Weights:

In [ ]:
generator.save_weights('/content/drive/MyDrive/License-super-resolution-master2/Weights/', save_format='tf')